In [ ]:
import pandas as pd
import glob
import deepl
from dotenv import load_dotenv
import os
load_dotenv() 

# df = pd.read_excel('data/nlmyo/processed/diag.xlsx')
# # Read the text content from the filename column and add it to a new column
# df["filepath"] = df["filename"].apply(lambda x: "data/nlmyo/processed/" + x)
# df['raw_text'] = df['filepath'].apply(lambda x: open(x, 'r').read())


# translator = deepl.Translator(os.getenv("DEEPL_KEY")) 
# df['deepl_translation'] = df['raw_text'].apply(lambda x: translator.translate_text(x, target_lang="EN-US").text)
# save df to csv file
# df.to_csv('data/nlmyo/processed/diag_translated.csv', index=False)

In [ ]:
import numpy as np
from embetter.external import CohereEncoder, OpenAIEncoder
from embetter.grab import ColumnGrabber
from sklearn.pipeline import make_pipeline 
import pandas as pd
from dotenv import load_dotenv
import os

from cohere import Client

# load_dotenv() 
# client = Client(os.getenv("COHERE_KEY"))
# # This pipeline grabs the `text` column from a dataframe

# df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')

# text_emb_pipeline = make_pipeline(
#   ColumnGrabber("deepl_translation"),
#   CohereEncoder(client=client, model="large")
# )

# X = text_emb_pipeline.fit_transform(df, df['diag_simple'])
# np.save('data/nlmyo/processed/report_translated_embed_cohere.npy', X)

In [ ]:
import numpy as np
from embetter.external import CohereEncoder, OpenAIEncoder
from embetter.grab import ColumnGrabber
from sklearn.pipeline import make_pipeline 
import pandas as pd
from dotenv import load_dotenv
import os

import openai

# load_dotenv() 

# openai.organization = os.getenv("OPENAI_ORG")
# openai.api_key = os.getenv("OPENAI_KEY")
# # This pipeline grabs the `text` column from a dataframe

# df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')

# text_emb_pipeline = make_pipeline(
#   ColumnGrabber("deepl_translation"),
#   OpenAIEncoder(model="text-embedding-ada-002")
# )

# X = text_emb_pipeline.fit_transform(df, df['diag_simple'])
# np.save('data/nlmyo/processed/report_translated_embed_openai.npy', X)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#### Import the data
df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')
Y = df['diag_simple'].values
X_cohere = np.load('data/nlmyo/processed/report_translated_embed_cohere.npy') 
X_openai = np.load('data/nlmyo/processed/report_translated_embed_openai.npy')

# Remove CFTD and unclear diagnosis
df['diag_simple'].value_counts()
df['diag_simple'] = df['diag_simple'].replace('CFTD', 'UNCLEAR')
# Drop the rows with unclear diagnosis
df = df[df['diag_simple'] != 'UNCLEAR']
Y = df['diag_simple'].values
# Do the same for the X array based on the df index
X_cohere = X_cohere[df.index]
X_openai = X_openai[df.index]
cv_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
clf_dummy = DummyClassifier(strategy='prior')
cv_scores_dummy = cross_val_score(clf_dummy, X_cohere, Y, cv=cv_fold)
print("Dummy Classifier Results:")
print(f"All CV Scores: {cv_scores_dummy}")
print(f"Mean CV Score:  {np.mean(cv_scores_dummy)}")
print(f"Standard Deviation CV Score: {np.std(cv_scores_dummy)}")

In [ ]:
clf = LogisticRegression(max_iter=3000)
cv_scores = cross_val_score(clf, X_cohere, Y, cv=cv_fold)
print("Results with Logistic Regression and Cohere Embeddings on English Translated Reports:")
print(f"All CV Scores: {cv_scores}")
print(f"Mean CV Score:  {np.mean(cv_scores)}")
print(f"Standard Deviation CV Score: {np.std(cv_scores)}")

In [ ]:
clf = LogisticRegression(max_iter=3000)
cv_scores = cross_val_score(clf, X_openai, Y, cv=cv_fold)
print("Results with Logistic Regression and OpenAI Embeddings on English Translated Reports:")
print(f"All CV Scores: {cv_scores}")
print(f"Mean CV Score:  {np.mean(cv_scores)}")
print(f"Standard Deviation CV Score: {np.std(cv_scores)}")

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass


# Create a pipeline
pipe = Pipeline([('clf', DummyEstimator())]) # Placeholder Estimator

# Candidate learning algorithms and their hyperparameters
search_space = [{'clf': [LogisticRegression()],
                    'clf__max_iter': [1500]},
                {'clf': [GaussianNB()],},
                {'clf': [MLPClassifier()],
                 'clf__max_iter': [500]},
                {'clf': [KNeighborsClassifier()],},
                {'clf': [SVC()],},
                {'clf': [GaussianProcessClassifier()],},
                {'clf': [HistGradientBoostingClassifier()],},
                {'clf': [DecisionTreeClassifier()],},
                {'clf': [RandomForestClassifier()],},
                {'clf': [AdaBoostClassifier()],},
                ]


# Create grid search 
gs = GridSearchCV(pipe, search_space, scoring="accuracy", cv=cv_fold)
gs.fit(X_cohere, Y)
df_cv_search = pd.DataFrame(gs.cv_results_)
df_cv_search.to_csv('data/nlmyo/processed/report_translated_embed_cohere_gridsearch.csv')
df_cv_search

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

# Create a pipeline
pipe = Pipeline([('clf', DummyEstimator())]) # Placeholder Estimator

# Candidate learning algorithms and their hyperparameters
search_space = [{'clf': [LogisticRegression()],
                    'clf__max_iter': [1500]},
                {'clf': [GaussianNB()],},
                {'clf': [MLPClassifier()],
                 'clf__max_iter': [1000]},
                {'clf': [KNeighborsClassifier()],},
                {'clf': [SVC()],},
                {'clf': [GaussianProcessClassifier()],},
                {'clf': [HistGradientBoostingClassifier()],},
                {'clf': [DecisionTreeClassifier()],},
                {'clf': [RandomForestClassifier()],},
                {'clf': [AdaBoostClassifier()],},
                ]


# Create grid search 
gs = GridSearchCV(pipe, search_space, scoring="accuracy", cv=cv_fold)
gs.fit(X_openai, Y)
df_cv_search = pd.DataFrame(gs.cv_results_)
df_cv_search.to_csv('data/nlmyo/processed/report_translated_embed_cohere_gridsearch.csv')
df_cv_search